In [2]:
import pygame
import sys

pygame.init()

# 화면 크기 설정
width, height = 800, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("라면 게임")

# 색상 및 폰트 설정
YELLOW = (255, 255, 0)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)

font_path = "C:/Windows/Fonts/malgun.ttf"
font = pygame.font.Font(font_path, 40)
small_font = pygame.font.Font(font_path, 30)

# 배경 이미지 설정
background_img = pygame.image.load("img.png")
background_img = pygame.transform.scale(background_img, (width, height))

# 점수 초기화
score = 0

# 이미지 로드 및 크기 조정
kettle_img = pygame.image.load("kettle.png")
kettle_img = pygame.transform.scale(kettle_img, (200, 200))

pot_img = pygame.image.load("pot.png")
pot_img = pygame.transform.scale(pot_img, (200, 200))  # 냄비는 좀 작게

# 게임 설명 이미지
instructions_img = pygame.image.load("First.png")
instructions_img = pygame.transform.scale(instructions_img, (width, height))

# 계란이미지
egg_img = pygame.image.load("egg.png")
egg_img = pygame.transform.scale(egg_img, (100, 100))

# 면 이미지
ramen_img = pygame.image.load("ramen.png")
ramen_img = pygame.transform.scale(ramen_img, (100, 100))

# 버튼 클래스
class Button:
    def __init__(self, x, y, width, height, text, action=None, color=(255, 232, 73)):
        self.rect = pygame.Rect(x, y, width, height)
        self.text = text
        self.action = action
        self.color = color

    def draw(self, screen):
        pygame.draw.rect(screen, self.color, self.rect)
        text_surface = font.render(self.text, True, BLACK)
        screen.blit(text_surface, (self.rect.centerx - text_surface.get_width() // 2,
                                   self.rect.centery - text_surface.get_height() // 2))

    def is_clicked(self, pos):
        return self.rect.collidepoint(pos)

# 라면 상태 및 드래그 처리 클래스 (이미지 추가)
class Block:
    def __init__(self, color=None, x=0, y=0, size=50, image=None):
        self.color = color
        self.rect = pygame.Rect(x, y, size, size)
        self.dragging = False
        self.image = image
        if image:
            self.rect.size = image.get_size()

    def draw(self, screen):
        if self.image:
            screen.blit(self.image, self.rect.topleft)
        else:
            pygame.draw.rect(screen, self.color, self.rect)

    def handle_event(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN and self.rect.collidepoint(event.pos):
            self.dragging = True
            mouse_x, mouse_y = event.pos
            self.offset_x = self.rect.x - mouse_x
            self.offset_y = self.rect.y - mouse_y
        elif event.type == pygame.MOUSEBUTTONUP:
            self.dragging = False
        elif event.type == pygame.MOUSEMOTION and self.dragging:
            mouse_x, mouse_y = event.pos
            self.rect.x = mouse_x + self.offset_x
            self.rect.y = mouse_y + self.offset_y

def show_score():
    global score
    score_text = font.render(f"점수: {score}원", True, BLACK)
    screen.blit(score_text, (20, 20))

def evaluate_ramen(cooking_time, correct_order):
    global score
    if not correct_order:
        score += 2000
    elif cooking_time >= 10:
        score += 0  # 불탐 상태로 실패
    elif cooking_time >= 4:
        score += 3000  # 적당한 상태
    elif cooking_time >= 2:
        score += 1000  # 미약 상태
    return True

def start_game():
    global score

    yellow_block = Block(image=egg_img, x=100, y=100)
    blue_block = Block(image=ramen_img, x=300, y=100)
    red_block = Block(RED, 500, 100)

    kettle = Block(image=kettle_img, x=450, y=height - 400)
    pot_list = [
        Block(image=pot_img, x=100, y=150),  # 왼쪽 위
        Block(image=pot_img, x=300, y=150),  # 오른쪽 위
        Block(image=pot_img, x=100, y=300),  # 왼쪽 아래
        Block(image=pot_img, x=300, y=300),  # 오른쪽 아래
    ]

    submit_button = Button(650, 500, 120, 50, "라면 제출", action=None)

    start_ticks = pygame.time.get_ticks()
    total_time = 60
    game_over = False
    cooking_time = 0
    correct_order = True

    while True:
        elapsed_time = (pygame.time.get_ticks() - start_ticks) / 1000
        remaining_time = max(0, total_time - elapsed_time)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

            if event.type == pygame.MOUSEBUTTONDOWN:
                if submit_button.is_clicked(event.pos):
                    evaluate_ramen(cooking_time, correct_order)
                    # 초기화
                    yellow_block = Block(image=egg_img, x=100, y=100)
                    blue_block = Block(image=ramen_img, x=300, y=100)
                    red_block = Block(RED, 500, 100)
                    kettle.rect.topleft = (600, height - 400)
                    for i, pot in enumerate(pot_list):
                        pot.rect.topleft = (300 + i * 120, height - 200)
                    cooking_time = 0

            yellow_block.handle_event(event)
            blue_block.handle_event(event)
            red_block.handle_event(event)
            kettle.handle_event(event)
            for pot in pot_list:
                pot.handle_event(event)

        screen.fill(WHITE)

        if not game_over:
            yellow_block.draw(screen)
            blue_block.draw(screen)
            red_block.draw(screen)
            kettle.draw(screen)
            for pot in pot_list:
                pot.draw(screen)

            submit_button.draw(screen)

            show_score()

        bar_width = 300
        bar_height = 30
        time_ratio = remaining_time / total_time
        pygame.draw.rect(screen, GRAY, (250, 20, bar_width, bar_height))
        pygame.draw.rect(screen, (233, 55, 113), (250, 20, int(bar_width * time_ratio), bar_height))

        if remaining_time <= 0 and not game_over:
            game_over = True

        if game_over:
            end_text = font.render("게임 종료!", True, RED)
            screen.blit(end_text, (width // 2 - end_text.get_width() // 2,
                                   height // 2 - end_text.get_height() // 2))

        pygame.display.update()

        if game_over:
            pygame.time.wait(3000)
            return

def show_instructions():
    screen.blit(instructions_img, (0, 0))

def loading_screen():
    start_button = Button(300, 250, 200, 50, "Start", start_game, color=(255, 232, 73))
    instructions_button = Button(300, 350, 200, 50, "게임 설명")
    game_button = Button(580, 500, 200, 50, "게임으로", start_game, color=(255, 232, 73))

    show_instructions_flag = False

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.MOUSEBUTTONDOWN:
                pos = event.pos
                if not show_instructions_flag:
                    if start_button.is_clicked(pos):
                        start_game()
                    elif instructions_button.is_clicked(pos):
                        show_instructions_flag = True
                else:
                    if game_button.is_clicked(pos):
                        start_game()
                    else:
                        # 화면 클릭 시 설명 화면 닫기
                        show_instructions_flag = False

        screen.blit(background_img, (0, 0))

        if not show_instructions_flag:
            start_button.draw(screen)
            instructions_button.draw(screen)
        else:
            show_instructions()
            game_button.draw(screen)

        pygame.display.update()

    pygame.quit()
    sys.exit()

# 실행
loading_screen()


SystemExit: 